## ДЗ выполнили:
Студенты группы _ИАД-1_
* Куприянов Кирилл
* Рахматуллин Айрат
* Данилин Павел

# Майнор "Прикладные задачи анализа данных"
## Домашнее задание 1 [10 баллов] до 23:59 17.02.2018

В этом домашнем задании вам потребуется написать генератор описания прогноза погоды на следующую неделю в каком-нибудь городе. Домашнее задание состоит из трех частей:
1. Скачивание данных о состоянии погоды в городе с gismeteo.ru
2. Генерация описания прогноза
3. Творческая часть

Все три части можно считать независимыми – вы можете сделать одну или две из них, однако мы настоятельно советуем выполнить все три. Все инструкции по выполнению домашнего задания – ниже. 



### 1. Сбор данных [3 балла]

Выберите произвольным образом город в России и найдите прогноз погоды в нем на ближайшие 10 дней на сайте gismeteo.ru.

Пример: прогноз на 10 ближайших дней в Москве – https://www.gismeteo.ru/weather-moscow-4368/10-days/

Используя известные вам библиотеки для работы с протоколом http и html кодом, извлеките прогноз на ближайшие 10 дней, начиная со дня, когда вы начали делать домашнее задание.  

Резльтатом сбора данных должна быть таблица со следующими строками:
* минимальная температура
* максимальна температура
* скорость ветра
* уровень осадков 

В столбцах таблицы должны быть даты и дни недели.  Пример итоговой таблицы вы найдете в следующей части задания. 

---

Сделаем import всех необходимых библиотек.

In [1]:
from scrapy.crawler import CrawlerProcess
from twisted.internet.error import ReactorNotRestartable
from json import dump

import scrapy
import random
import pandas as pd
import datetime
import re
import pymorphy2

Выберем случайный российский город. Данные взяты из [Википедии](https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_Russia).

In [2]:
CITIES_URL = 'https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_Russia'
CITY_SELECTOR = '//tr/td[2]'

In [3]:
cities = []
class RussianCities(scrapy.Spider):
    name = 'russian_cities'
    start_urls = [CITIES_URL,]
    def parse(self, response):
        for city in response.xpath(CITY_SELECTOR):
            cities.append(city.xpath('text()').extract_first())

In [4]:
process = CrawlerProcess()
process.crawl(RussianCities)
process.start()

2018-02-17 18:56:46 [scrapy.utils.log] INFO: Scrapy 1.5.0 started (bot: scrapybot)
2018-02-17 18:56:46 [scrapy.utils.log] INFO: Versions: lxml 4.1.1.0, libxml2 2.9.7, cssselect 1.0.3, parsel 1.4.0, w3lib 1.19.0, Twisted 17.9.0, Python 2.7.14 (default, Jan  5 2018, 10:41:29) - [GCC 7.2.1 20171224], pyOpenSSL 17.5.0 (OpenSSL 1.1.0g  2 Nov 2017), cryptography 2.1.4, Platform Linux-4.14.15-gnu-1-x86_64-with-glibc2.2.5
2018-02-17 18:56:46 [scrapy.crawler] INFO: Overridden settings: {}
2018-02-17 18:56:46 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.corestats.CoreStats']
2018-02-17 18:56:46 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddlew

In [5]:
len(cities)

1112

In [6]:
random.seed(4118956)
print(random.choice(cities))

Сергиев Посад


Выбрали Сергиев Посад. 

Если пробовать запустить дальше код, будет кидаться ошибка __ReactorNotRestartable__, поэтому:

Для воспроизведения кода далее необходимо перезагрузить ядро (kernel $\rightarrow$ restart) и начать выполнение с _этой_  клетки.
Дело в том, что дальше запускается ещё один spider scrapy, а два twisted process'a не могут одновременно существовать (поскольку предыдущий уже запущен).

In [1]:
from scrapy.crawler import CrawlerProcess
from twisted.internet.error import ReactorNotRestartable
from json import dump

import scrapy
import random
import pandas as pd
import datetime
import re
import pymorphy2

In [2]:
CITY_URL = 'https://www.gismeteo.ru/weather-sergiyev-posad-11958/10-days/'

# Выделим все селекторы в словарь, чтобы при их изменении 
# на сайте, можно было поменять их здесь и один раз.
#
SELECTORS = {
    'weekday': '//div[@class="forecast_frame"]//div[@class="w_date__day"]/text()',
    'day': '//span[contains(@class, "w_date__date")]/text()',
    'min_temp': '//div[@class="maxt"]/text()',
    'max_temp': '//div[@class="mint"]/text()',
    'wind_speed': '//div[contains(@class,"w_wind__warning")]/text()',
    'precipitation': '//div[contains(@class,"w_precipitation__value")]/text()',
}

def process_digit(d):
    '''
    Removes all junk like \n and whitespaces.
    Returns: pure float number.
    '''
    return re.findall('[\d,]+', d)[0]

Данные о погоде будут собираться по состоянию на 12.02.2018

In [3]:
json_data = []
class GismeteoForecast(scrapy.Spider):
    name = 'gismeteo_forecast'
    start_urls = [CITY_URL,]
    def parse(self, response):
        weekdays = response.xpath(SELECTORS['weekday']).extract()
        days = response.xpath(SELECTORS['day']).extract()[:10]
        min_temp = response.xpath(SELECTORS['min_temp']).extract()[:10]
        max_temp = response.xpath(SELECTORS['max_temp']).extract()[:10]
        wind_speed =  response.xpath(SELECTORS['wind_speed']).extract()[:10]
        precipitation = response.xpath(SELECTORS['precipitation']).extract()
        for i in range(10):
            json_data.append({
                'weekday': weekdays[i],
                'day': process_digit(days[i]),
                'minTemp': min_temp[i],
                'maxTemp': max_temp[i],
                'windSpeed': process_digit(wind_speed[i]),
                'precipitation': process_digit(precipitation[i]),
            })

In [4]:
# Этот блок try-except на случай если вы забыли перезапустить ядро..
#
try:
    process = CrawlerProcess()
    process.crawl(GismeteoForecast)
    process.start()
except ReactorNotRestartable:
    print('\n\n\n------------\nRestart kernel and rerun the cells where it\'s said to do so (see above).\n------------')

2018-02-17 18:57:12 [scrapy.utils.log] INFO: Scrapy 1.5.0 started (bot: scrapybot)
2018-02-17 18:57:12 [scrapy.utils.log] INFO: Versions: lxml 4.1.1.0, libxml2 2.9.7, cssselect 1.0.3, parsel 1.4.0, w3lib 1.19.0, Twisted 17.9.0, Python 2.7.14 (default, Jan  5 2018, 10:41:29) - [GCC 7.2.1 20171224], pyOpenSSL 17.5.0 (OpenSSL 1.1.0g  2 Nov 2017), cryptography 2.1.4, Platform Linux-4.14.15-gnu-1-x86_64-with-glibc2.2.5
2018-02-17 18:57:12 [scrapy.crawler] INFO: Overridden settings: {}
2018-02-17 18:57:12 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.corestats.CoreStats']
2018-02-17 18:57:12 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddlew

Сохраним результаты в json.

In [5]:
with open('result.json', 'w') as fp:
    dump(json_data, fp)

Считаем в dataframe записанный json-файл.

In [6]:
df = pd.read_json('result.json')

In [7]:
df

,day,maxTemp,minTemp,precipitation,weekday,windSpeed
0,17,−6,−4,"0,55",Сб,9
1,18,−7,−4,"3,1",Вс,9
2,19,−7,−3,"1,2",Пн,5
3,20,−8,−6,"0,6",Вт,7
4,21,−12,−9,"0,7",Ср,10
5,22,−15,−11,0,Чт,7
6,23,−16,−10,0,Пт,5
7,24,−17,−11,"0,4",Сб,9
8,25,−18,−15,0,Вс,11
9,26,−20,−17,"0,6",Пн,10


Зададим таблице правильный индекс и транспонируем.

In [8]:
days = df['day']
weekdays = df['weekday']
new_index = []

for day, w_day in zip(days, weekdays):
    new_index.append('{d}.0{m} ({w})'.format(d=day,
                                            w=w_day.encode('utf-8'),
                                            m=datetime.date.today().month))

df.drop(['day', 'weekday'], axis=1, inplace=True)
df.index = new_index
df = df.T

In [9]:
df

,17.02 (Сб),18.02 (Вс),19.02 (Пн),20.02 (Вт),21.02 (Ср),22.02 (Чт),23.02 (Пт),24.02 (Сб),25.02 (Вс),26.02 (Пн)
maxTemp,−6,−7,−7,−8,−12,−15,−16,−17,−18,−20
minTemp,−4,−4,−3,−6,−9,−11,−10,−11,−15,−17
precipitation,"0,55","3,1","1,2","0,6","0,7",0,0,"0,4",0,"0,6"
windSpeed,9,9,5,7,10,7,5,9,11,10


### 2. Генератор описания прогноза погоды [4 балла]

Если у вас не получилось извлечь прогноз погоды в предыдущей части задания, воспользуйтесь таблицей ниже.
В ней приведен прогноз четырех показателей на ближайшие 10 дней в Москве – минимальная и максимальная температура, скорость ветра и уровень осадков. 

|                | 02.02 (пт) | 03.02 (сб) | 04.02 (вс)| 05.02 (пн) | 06.02 (вт) | 07.02 (пн) | 08.02 (ср) | 09.02 (ср) | 10.02 (сб) | 11.02 (вс)
|----------------|-------|-------|-------|-------|-------|-------|-------|
| минимальная температура    | -9    | -1    | -8    | -13    | -12    | -15    | -21    | -14 |-8 |-8
| максимальная температура    | -1    | +1    | -2    | -9   | -11    | -12    | -16    |-5    |-6    |-5|
| скорость ветра | 10    | 13    | 15    | 15   |11    | 6    | 7 | 9 | 8 |12
| уровень осадков         | 1.35  | 8.6  | 15.5  | 6.6   | 2.7   | 2.1   | 0   | 3.2   |0.8  | 0.4

Прогноз погоды должен состоять из следующих (или подобным им) предложений, генерируемых по шаблонам (ниже три шаблона):
* В день1 похолодает / потеплеет на X градус (-а, -ов) по сравнению с день2
    * *В четверг в НазваниеГорода потеплеет на 7 градусов по сравнению со средой*
* Скорость ветра изменится на X единиц в день1 по сравнению с день2.
    * *Скорость ветра изменится на 3 единицы в понедельник по сравнению с пятницей*
* Уровень осадков повысится / понизится на X единиц за Y дней. 
    * *Уровень осадков понится на 3.85 единиц за 7 дней*

Вместо НазваниеГорода подставьте название выбранного вами города, используя фунцкии для согласования существительных с предлогами. 

Некоторые вспомогательные функции, которые вам понадобятся: согласование существительного с числительным и приведение существительного к нужному падежу: 

In [10]:
df = pd.read_csv('gismeteo_forecast.csv', sep=';', index_col=0)

In [11]:
df.head()

,13.02 (Вт),14.02 (Ср),15.02 (Чт),16.02 (Пт),17.02 (Сб),18.02 (Вс),19.02 (Пн),20.02 (Вт),21.02 (Ср),22.02 (Чт)
maxTemp,−13,−14,−13,−8,−5,−4,−3,−2,−4,−9
minTemp,−7,−8,−8,−4,−2,−1,−1,+3,+3,−4
precipitation,0,0,"0,2",3,"3,8","0,8","0,8","8,8","0,2",0
windSpeed,7,8,7,10,10,8,9,15,15,5


In [12]:
name_of_city = u'SOME_CITY_NAME'

In [13]:
months_names = (u'января', u'февраля', u'марта', u'апреля', u'мая', u'июня', 
                u'июля', u'августа', u'сентября', u'октября', u'ноября', u'декабря')

In [14]:
morph = pymorphy2.MorphAnalyzer()

2018-02-17 18:57:24 [pymorphy2.opencorpora_dict.wrapper] INFO: Loading dictionaries from /usr/lib/python3.6/site-packages/pymorphy2_dicts/data
2018-02-17 18:57:24 [pymorphy2.opencorpora_dict.wrapper] INFO: format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


In [15]:
def create_sentences(df, cols_name):
    # В день1 похолодает / потеплеет на X градус (-а, -ов) по сравнению с день2
    # Скорость ветра изменится на X единиц в день1 по сравнению с день2.
    # Уровень осадков повысится / понизится на X единиц за Y дней.
    dates = [i.split()[0] for i in cols_name]
    days = [i.split('.')[0] for i in dates]
    months = (months_names[int(i.split('.')[1]) - 1] for i in dates)
    dates = [day + ' ' + month for day, month in zip(days, months)]
    cols = [df[i].apply(lambda x: x.replace('−', '-').replace(',', '.')) for i in cols_name]

    changes = {i : float(cols[1][i]) - float(cols[0][i]) for i in df.index}        
        
    def get_changing_str(x, units):
        changing_str = None
        if x == 0:
            changing_str = u'не изменится'
        else:
            if x > 0:
                changing_str = u'повысится'
            else:
                changing_str = u'понизится'
            changing_str += u' на {abs} {morph}'\
                            .format(abs=abs(x),
                                    morph=morph.parse(units)[0].make_agree_with_number(abs(x)).word)

        return changing_str

    create_temp_str = lambda x, s:\
    u'{d1} {s} температура {deg} по сравнению с {d2}'.format(d1=dates[1],
                                                            s=s,
                                                            deg=get_changing_str(int(x), u'градус'),
                                                            d2=dates[0])
        
    create_precip = lambda x:\
    u'В период с {d1} по {d2} уровень осадков {deg}'.format(d1=dates[0],
                                                           d2=dates[1],
                                                           deg=get_changing_str(round(x, 1), ''))
    
    
    print(create_temp_str(changes['maxTemp'], u'максимальная'))
    print(create_temp_str(changes['minTemp'], u'минимальная'))

    print(u'Скорость ветра изменится на {wind_change} {morph} {d1} по сравнению с {d2}'\
          .format(wind_change=int(changes['windSpeed']),
                  morph=morph
                       .parse(u'единица')[0]
                       .inflect({'accs'})
                       .make_agree_with_number(abs(int(changes['windSpeed'])))
                       .word,
                  d2=dates[0],
                  d1=dates[1]))
    print(create_precip(changes['precipitation']))

    
create_sentences(df, (df.columns[0], df.columns[1]))

14 февраля максимальная температура понизится на 1 градус по сравнению с 13 февраля
14 февраля минимальная температура понизится на 1 градус по сравнению с 13 февраля
Скорость ветра изменится на 1 единицу 14 февраля по сравнению с 13 февраля
В период с 13 февраля по 14 февраля уровень осадков не изменится


In [16]:
create_sentences(df, (df.columns[0], df.columns[1]))

14 февраля максимальная температура понизится на 1 градус по сравнению с 13 февраля
14 февраля минимальная температура понизится на 1 градус по сравнению с 13 февраля
Скорость ветра изменится на 1 единицу 14 февраля по сравнению с 13 февраля
В период с 13 февраля по 14 февраля уровень осадков не изменится


In [17]:
for ind_i, i in enumerate(df.columns[:-1]):
    for j in df.columns[ind_i + 1:]:
        create_sentences(df, (i, j))

14 февраля максимальная температура понизится на 1 градус по сравнению с 13 февраля
14 февраля минимальная температура понизится на 1 градус по сравнению с 13 февраля
Скорость ветра изменится на 1 единицу 14 февраля по сравнению с 13 февраля
В период с 13 февраля по 14 февраля уровень осадков не изменится
15 февраля максимальная температура не изменится по сравнению с 13 февраля
15 февраля минимальная температура понизится на 1 градус по сравнению с 13 февраля
Скорость ветра изменится на 0 единиц 15 февраля по сравнению с 13 февраля
В период с 13 февраля по 15 февраля уровень осадков повысится на 0.2 
16 февраля максимальная температура повысится на 5 градусов по сравнению с 13 февраля
16 февраля минимальная температура повысится на 3 градуса по сравнению с 13 февраля
Скорость ветра изменится на 3 единицы 16 февраля по сравнению с 13 февраля
В период с 13 февраля по 16 февраля уровень осадков повысится на 3.0 
17 февраля максимальная температура повысится на 8 градусов по сравнению с 1

### 3. Ответьте на вопросы [3 балла]
**Вопрос:** В каких других задачах (помимо описания прогноза погоды) может понадобиться генерировать текст по шаблонам?

**Ответ:** 
1. Генерация текста по шаблонам может понадобиться в следующих случаях:
1. Автоматические новости на новостных сайтах
1. Автоматические описания товаров на сайтах интернет магазинов
1. Реклама однотипных товаров
1. Автоматическая служба поддержки
1. Бот, общающийся с пользователем на естественном языке
1. Описание изменений курса валют
1. Бегущая строка в метро
1. Поздравления с праздниками в корпоративных помещениях

**Вопрос:** В каких задачах может понадобиться генерировать текст об изменении числовых показателей по шаблонам?

**Ответ:**
1. Описание изменений курса валют
1. Описание состояния показателей производства (производительность, динамика эффективности)
1. Генерация отчетов по работе предприятия
1. Генерация отчетов по экономических показателям

**Вопрос:** Шаблоны, которые вы использовали в этом задании, имеют фиксированную структуру. Фактически, ваша задача заключалась в том, чтобы подставить в шаблон число и согласовать единицы измерения с этим числом или подставить в шаблон название города и согласовать его с предлогом. Как можно разнообразить эти шаблоны? Как знание синтаксической структуры предложения может помочь в этой задаче? 

**Ответ:** Можно добавить новые шаблоны. Или например, динамически формировать полностью предложения вместо некоторых подстановок. Знание синтаксической структуры позволяет достаточно легко осуществлять подстановки, согласованные с остальными словами предложения.

EOF